In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import skimage.io as skio
from skimage.transform import resize
import pandas as pd
import os
import requests
from pyarrow import feather
import io
import concurrent.futures

import warnings
warnings.filterwarnings("ignore")

In [ ]:
def convert_audio_to_mel_image(filepath_to_audio, filepath_to_save, image_size=(128,512), n_mels=128, fmax=8000,):
    signal, sr = librosa.load(filepath_to_audio)
    mels = librosa.power_to_db(librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=n_mels, fmax=fmax), ref=np.max)
    image = (((80+mels)/80)*255)
    image = np.flip(image, axis=0)
    resize_image = resize(image, image_size).astype(np.uint8)
    skio.imsave(filepath_to_save, resize_image)
    
def convert_audio_to_composite_image(filepath_to_audio, filepath_to_save, image_size=(128,512), n_mels=128, fmax=8000,):
    
    signal, sr = librosa.load(filepath_to_audio)
    
    mels = librosa.power_to_db(librosa.feature.melspectrogram(y=signal, sr=sr, n_mels=n_mels, fmax=fmax), ref=np.max)
    mel_image = (((80+mels)/80)*255)
    mel_image = np.flip(mel_image, axis=0)
    mel_image = resize(mel_image, (128,512)).astype(np.uint8)
    
    mfcc = librosa.power_to_db(librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=128, fmax=8000), ref=np.max)
    mfcc_image = (((80+mfcc)/80)*255)
    mfcc_image = np.flip(mfcc_image, axis=0)
    mfcc_image = resize(mfcc_image, (128,512)).astype(np.uint8)
    
    chromagram = librosa.feature.chroma_cqt(y=signal, sr=sr)
    chroma_image = resize(chromagram*255, (128,512)).astype(np.uint8)
    
    composite = np.dstack((mel_image, mfcc_image, chroma_image))
    
    skio.imsave(filepath_to_save, composite)

In [ ]:
tracks_df = feather.read_feather('data/all_tracks.feather')

In [ ]:
tracks_df = tracks_df.dropna(subset=['track_preview_link'])

In [ ]:
random_tracks_df = tracks_df.sample(frac=1, random_state=42).reset_index(drop=True)
random_tracks_df

In [ ]:
def download_preview_with_index(index):
    track_url = random_tracks_df.loc[index, 'track_preview_link']

    preview = requests.get(track_url)
    
    track_id = random_tracks_df.loc[index, 'track_id']

    filename = f'data/Spotify/mp3s/{track_id}.mp3'
    png_name = f'data/Spotify/comp_pngs/{track_id}.png'

    with open(filename, 'wb') as f:
        f.write(preview.content)

    convert_audio_to_composite_image(filename, png_name)

    os.remove(filename)
    
    print(index, end='\r')
    

In [ ]:
pd.options.mode.chained_assignment = None

indices = tracks_df.index
MAX_THREADS = 50
threads = min(MAX_THREADS, len(indices))
count = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    executor.map(download_preview_with_index, indices)